In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import pandas as pd
import numpy as np

import time

import torch

from tqdm.auto import tqdm

In [2]:
from datasets import load_original_dataset, load_deleted_dataset
from models import CNN

In [3]:
DATA_DIR = 'Datasets/Features/'
BATCH_SIZE = 32
EPOCHS = 5
PERCENTAGES = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 99]

In [4]:
def fit(model, save_dir, train_set, test_set, forget_set=None):
    
    os.makedirs(save_dir, exist_ok=True)
    
    optimizer = torch.optim.Adam(model.parameters())
    error = torch.nn.CrossEntropyLoss()
    
    train_loader = torch.utils.data.DataLoader(train_set, batch_size = BATCH_SIZE, shuffle = True, drop_last=True)
    
    train_times = list()
    train_accs, test_accs, forget_accs = list(), list(), list()
    
    for epoch in range(EPOCHS):
        
        # train
        
        accs = list()
        
        train_time = 0
        
        model.train()
        
        start_time = time.time()
        
        for x, y in train_loader:
            optimizer.zero_grad()
            output = model(x.cuda())
            y = y.cuda()
            loss = error(output, y)
            loss.backward()
            optimizer.step()
            
            train_time += time.time() - start_time
            
            predicted = torch.argmax(output.data, dim=-1)
            accs.append((predicted == y).float().mean().detach().cpu().numpy())
            
            start_time = time.time()
            
        train_times.append(train_time)
        train_accs.append(np.mean(accs))
        
        # eval
            
        model.eval()
        with torch.no_grad():
            
            x, y = test_set.tensors
            
            accs = list()
            
            for i in range(0, x.shape[0], BATCH_SIZE):
            
                output = model(x[i:i+BATCH_SIZE].cuda())

                predicted = torch.argmax(output.data, dim=-1)
                accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())
            
            test_accs.append(np.mean(accs))
            
            if forget_set is not None:

                x, y = forget_set.tensors

                for i in range(0, x.shape[0], BATCH_SIZE):

                    output = model(x[i:i+BATCH_SIZE].cuda())

                    predicted = torch.argmax(output.data, dim=-1)
                    accs.append((predicted == y[i:i+BATCH_SIZE].cuda()).float().mean().detach().cpu().numpy())

                forget_accs.append(np.mean(accs))
        
        # save
        torch.save(model.state_dict(), os.path.join(save_dir, f'{(epoch+1):03d}.pt'))
        

    return train_times, train_accs, test_accs, forget_accs

# origin train

In [5]:
train_set, test_set = load_original_dataset(DATA_DIR)

In [6]:
model = CNN().cuda()

In [7]:
train_times, train_accs, test_accs, _ = fit(model, 'weights/original', train_set, test_set)

In [8]:
df = pd.DataFrame(zip(train_times, train_accs, test_accs), columns=['train_time', 'train_acc', 'test_acc'])
df['epoch'] = range(1, EPOCHS+1)
df.set_index('epoch', inplace=True)

df.to_csv('results/original.csv')

df

,train_time,train_acc,test_acc
epoch,,,
1,12.081916,0.889200,0.981829
2,10.591890,0.950167,0.985523
3,10.323363,0.953600,0.985224
4,10.458335,0.958050,0.985423
5,10.305062,0.959650,0.987121


# naive retrain

In [10]:
results = list()

for percentage in tqdm(PERCENTAGES):
    
    model = CNN().cuda()
    
    train_set, test_set, forget_set = load_deleted_dataset(DATA_DIR, percentage)
    
    train_times, train_accs, test_accs, forget_accs = fit(model, f'weights/naive_retrain/{percentage}', train_set, test_set, forget_set)
    
    df = pd.DataFrame(zip(train_times, train_accs, test_accs, forget_accs), columns=['train_time', 'train_acc', 'test_acc', 'forget_acc'])
    df['epoch'] = range(1, EPOCHS+1)
    df['percentage'] = percentage
    
    results.append(df)

  0%|          | 0/11 [00:00<?, ?it/s]

In [11]:
results = pd.concat(results).set_index(['percentage', 'epoch'])

results.to_csv('results/naive_retrain.csv')

results

train_time  train_acc  test_acc  forget_acc
percentage epoch                                             
1          1       10.728109   0.858129  0.972943    0.972609
           2       10.466601   0.939824  0.978634    0.978257
           3       10.605615   0.951189  0.983027    0.982869
           4       10.338984   0.954826  0.985423    0.985505
           5       10.454302   0.956290  0.986422    0.986069
10         1        9.334682   0.823781  0.974341    0.974052
           2        9.642615   0.938371  0.980232    0.979915
           3        9.373440   0.950022  0.983127    0.983408
           4        9.440527   0.953431  0.984225    0.983408
           5        9.214355   0.957395  0.987021    0.986402
20         1        8.321174   0.868646  0.974441    0.973383
           2        8.475423   0.946521  0.984425    0.982649
           3        8.419238   0.952354  0.987720    0.985329
           4        8.513315   0.957833  0.985423    0.984829
           5        8.316424   0.958458  0.986322    0.984784
30         1        7.425974   0.831555  0.972843    0.969535
           2        7.706931   0.938477  0.983327    0.980629
           3        6.984120   0.946480  0.984625    0.982663
           4        7.360010   0.951791  0.983327    0.982199
           5        7.350313   0.955936  0.984325    0.981414
40         1        6.024324   0.836222  0.971446    0.970102
           2        6.313870   0.939333  0.980731    0.978010
           3        6.281009   0.949444  0.978435    0.978745
           4        6.202508   0.952361  0.984625    0.983126
           5        6.286266   0.955000  0.986621    0.984684
50         1        5.286074   0.809565  0.970447    0.968300
           2        5.258087   0.932130  0.975639    0.975619
           3        5.392960   0.945338  0.981130    0.978192
           4        5.227308   0.950140  0.982328    0.980740
           5        5.314469   0.953975  0.985823    0.982914
60         1        4.238559   0.784250  0.960463    0.958102
           2        4.219870   0.923792  0.972344    0.970445
           3        4.118789   0.938542  0.977835    0.976552
           4        4.068602   0.944125  0.979932    0.977443
           5        4.257469   0.949125  0.984026    0.980746
70         1        3.082684   0.744106  0.959365    0.956757
           2        3.110268   0.917482  0.969149    0.965675
           3        3.200110   0.933218  0.975739    0.973555
           4        2.941157   0.939224  0.980232    0.976399
           5        3.130415   0.943950  0.977736    0.974054
80         1        1.981479   0.713833  0.947484    0.943171
           2        2.013653   0.909083  0.967053    0.964630
           3        1.942061   0.927083  0.975040    0.972870
           4        2.105720   0.938833  0.973243    0.969715
           5        2.039008   0.947333  0.979333    0.975817
90         1        0.932643   0.560662  0.912041    0.902314
           2        0.915291   0.863636  0.933606    0.927474
           3        0.965449   0.904412  0.948882    0.942497
           4        1.005918   0.919953  0.959864    0.953633
           5        0.972820   0.932320  0.967851    0.962612
99         1        0.062696   0.119792  0.166334    0.159879
           2        0.071280   0.168403  0.371705    0.367612
           3        0.084393   0.333333  0.466554    0.458525
           4        0.070013   0.468750  0.565395    0.563969
           5        0.069151   0.590278  0.696086    0.688148